In [2]:
#import packages

import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#set directories for input and output

IN_DIR = "/Users/adarsh_23/CNS/Envisioning-Kaggle-main/Metadata"
OUT_DIR = "/Users/adarsh_23/CNS/Envisioning-Kaggle-main/Filtered"


In [6]:
#read in dfs and declare variables to select competition of interest
comps_df = pd.read_csv(IN_DIR+'/Competitions.csv', parse_dates=['DeadlineDate'])
#comps_df.shape
comp_title = "HuBMAP + HPA - Hacking the Human Body"
comp_overview = comps_df[comps_df.Title==comp_title]
comp_id = comp_overview.iloc[0]['Id']
#comp_id
comp_forum_id = comp_overview.iloc[0]['ForumId']
comp_forum_id

1960467.0

In [7]:
comp_overview

,Id,Slug,Title,Subtitle,HostSegmentTitle,ForumId,OrganizationId,CompetitionTypeId,HostName,EnabledDate,...,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,RewardType,RewardQuantity,NumPrizes,UserRankMultiplier,CanQualifyTiers,TotalTeams,TotalCompetitors,TotalSubmissions
5254,34547,hubmap-organ-segmentation,HuBMAP + HPA - Hacking the Human Body,Segment multi-organ functional tissue units,Research,1960467.0,3808.0,1,NaN,06/22/2022 18:34:11,...,False,False,USD,60000.0,3,1.0,True,1175,1517,39568


In [8]:
#write filtered competitions dataframe to csv
comp_overview.to_csv(OUT_DIR+'/Competitions.csv')

In [9]:
# filter and write Teams
teams_df = pd.read_csv(IN_DIR+'/Teams.csv', infer_datetime_format=True)

comp_teams_df = teams_df[teams_df.CompetitionId==comp_id].dropna(subset=['LastSubmissionDate'])

team_ids_list = comp_teams_df.Id.unique().tolist()
#len(team_ids_list)

comp_teams_df.to_csv(OUT_DIR+'/Teams.csv')

/var/folders/hc/1stky0sd2_z9nqk66bk22rs80000gn/T/ipykernel_53778/3526095410.py:2: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  teams_df = pd.read_csv(IN_DIR+'/Teams.csv', infer_datetime_format=True)


In [10]:
#filter forums, forumtopics, forumMessages, and forum Message Votes
forums_df = pd.read_csv(IN_DIR+'/Forums.csv', infer_datetime_format=True)
forums_topics_df = pd.read_csv(IN_DIR+'/ForumTopics.csv', infer_datetime_format=True)
forums_messages_df = pd.read_csv(IN_DIR+'/ForumMessages.csv', infer_datetime_format=True)
forums_message_votes_df = pd.read_csv(IN_DIR+'/ForumMessageVotes.csv', infer_datetime_format=True)

In [11]:
comp_forums_df = forums_df[forums_df.Id==comp_forum_id]
comp_forums_topics_df = forums_topics_df[forums_topics_df.ForumId==comp_forum_id]

#have to filter by forum topic
topic_ids = list(comp_forums_topics_df.Id.unique())
#topic_ids

comp_forums_messages_df = forums_messages_df[forums_messages_df.ForumTopicId.isin(topic_ids)]
#comp_forums_messages_df.head(5)

#have to filter by forum message
message_ids = list(comp_forums_messages_df.Id.unique())
comp_forums_message_votes_df = forums_message_votes_df[forums_message_votes_df.ForumMessageId.isin(message_ids)]
comp_forums_message_votes_df.head(5)

,Id,ForumMessageId,FromUserId,ToUserId,VoteDate
2094931,2834499,1829591,3012786,1095143,06/22/2022
2094932,2834523,1829591,8404799,1095143,06/22/2022
2094933,2846267,1829591,10941367,1095143,06/29/2022
2094934,2850954,1829591,10731811,1095143,07/02/2022
2094935,2913336,1829591,11071926,1095143,08/02/2022


In [12]:
comp_forums_messages_df.shape


(2149, 8)

In [13]:
comp_forums_df.to_csv(OUT_DIR+'/Forums.csv')
comp_forums_topics_df.to_csv(OUT_DIR+'/ForumTopics.csv')
comp_forums_messages_df.to_csv(OUT_DIR+'/ForumMessages.csv')
comp_forums_message_votes_df.to_csv(OUT_DIR+'/ForumMessageVotes.csv')

In [14]:
#Filter users by whether they are on a team in the competition or not
users_df = pd.read_csv(IN_DIR+'/Users.csv', infer_datetime_format=True)
#for this, I will also need the teammemberships info
team_memberships_df = pd.read_csv(IN_DIR+'/TeamMemberships.csv', infer_datetime_format=True)
team_memberships_df.shape


(5908972, 4)

In [15]:
comp_team_memberships_df = team_memberships_df[team_memberships_df.TeamId.isin(team_ids_list)]
comp_team_memberships_df.head(5)

,Id,TeamId,UserId,RequestDate
5813492,9069105,8805826,2132795,06/22/2022
5813497,9069126,8805846,1107630,06/22/2022
5813499,9069134,8805854,6537187,06/22/2022
5813505,9069170,8805890,10654180,06/22/2022
5813506,9069174,8805894,4212496,06/22/2022


In [18]:
comp_users_list = list(comp_team_memberships_df.UserId.unique())
#comp_users_list
comp_users_df = users_df[users_df.Id.isin(comp_users_list)]
comp_users_df.to_csv(OUT_DIR+'/Users.csv')


In [19]:
kernelVersionCompSources_df = pd.read_csv(IN_DIR+'/KernelVersionCompetitionSources.csv', infer_datetime_format=True)

In [20]:
comp_kernelversionids_df = kernelVersionCompSources_df[kernelVersionCompSources_df.SourceCompetitionId==comp_id]
comp_kernelversionids_df.shape

(8248, 3)

In [21]:
comp_kernel_ids_list = comp_kernelversionids_df.KernelVersionId.unique()
len(comp_kernel_ids_list)

8248

In [22]:
kernels_df = pd.read_csv(IN_DIR+'/Kernels.csv', infer_datetime_format=True)
kernels_df.shape

(808059, 16)

In [23]:
comp_kernels_df = kernels_df[kernels_df.CurrentKernelVersionId.isin(comp_kernel_ids_list)]
#comp_kernels_df.shape (350,16)
comp_kernels_df.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
604495,22725748,8659751,104435323.0,82118583.0,NaN,82121236.0,12/11/2021 15:02:31,08/29/2022,07/23/2022,False,hub-inference-single,NaN,NaN,250,0,5
736679,28160803,10654180,99073075.0,NaN,332690.0,99068065.0,06/22/2022 19:23:38,06/22/2022,06/22/2022,False,hubmap-quick-eda,2.0,09/04/2022,579,1,26
736684,28160898,1536542,99068725.0,NaN,342546.0,99068387.0,06/22/2022 19:30:46,06/22/2022,06/22/2022,False,background-information,2.0,07/14/2022,1949,0,41
736687,28161018,3894504,100803535.0,NaN,336792.0,99068759.0,06/22/2022 19:38:17,07/14/2022,06/26/2022,False,eda-simplified-hubmap-hpa-organ-segmentation,3.0,07/13/2022,207,2,9
736690,28161123,1314864,99071779.0,NaN,333397.0,99069092.0,06/22/2022 19:45:55,06/22/2022,06/22/2022,False,hubmap-hpa-exploratory-data-analysis,2.0,06/29/2022,560,6,36


In [24]:
comp_kernels_df.shape

(350, 16)

In [25]:
comp_kernels_df.to_csv(OUT_DIR+'/Kernels.csv')


In [26]:
comp_kernels_df.shape


(350, 16)

In [29]:
#filter datasets by forum id: 995088.0

datasets_df = pd.read_csv(IN_DIR+'/Datasets.csv')
print(datasets_df.shape) #(183770, 14)
datasets_df.head()

(183770, 14)


,Id,CreatorUserId,OwnerUserId,OwnerOrganizationId,CurrentDatasetVersionId,CurrentDatasourceVersionId,ForumId,Type,CreationDate,LastActivityDate,TotalViews,TotalDownloads,TotalVotes,TotalKernels
0,10,993,NaN,4.0,793490.0,815288.0,798,2,09/11/2015 01:56:00,02/06/2018,187794,17583,292,340
1,12,993,993.0,NaN,2156.0,2156.0,966,2,12/09/2015 06:16:00,02/04/2018,74682,4678,99,81
2,35,259561,259561.0,NaN,58.0,58.0,1262,2,05/05/2016 23:11:34,02/06/2018,307787,42773,712,263
3,37,366532,NaN,854.0,793647.0,815446.0,1277,2,05/13/2016 20:05:17,02/05/2018,97093,8133,134,100
4,42,304806,304806.0,NaN,799955.0,821873.0,1289,2,05/19/2016 23:46:20,02/04/2018,48836,3260,91,39


In [30]:
comp_datasets_df = datasets_df[datasets_df.ForumId==int(comp_forum_id)]
comp_datasets_df.shape

(0, 14)

In [31]:
int(comp_forum_id)

1960467

In [32]:
print(datasets_df.ForumId.unique())

[    798     966    1262 ... 2585276 2585345 2585560]


In [33]:
# troubleshooting kernal lineage network
kernels_df.head()


,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
0,1,2505,205.0,NaN,NaN,1.0,03/25/2015 18:25:32,03/23/2018,03/25/2015,False,hello,NaN,NaN,306,0,0
1,2,3716,1748.0,NaN,26670.0,2.0,03/25/2015 18:31:07,04/16/2015,03/25/2015,False,rf-proximity,3.0,07/15/2016,8867,1,13
2,4,3716,41.0,NaN,NaN,9.0,03/25/2015 21:57:36,03/23/2018,03/25/2015,False,r-version,NaN,NaN,138,0,0
3,5,28963,19.0,NaN,NaN,13.0,03/25/2015 22:01:04,03/23/2018,03/25/2015,False,test1,NaN,NaN,130,0,0
4,6,3716,21.0,NaN,NaN,15.0,03/25/2015 22:19:00,03/23/2018,03/25/2015,False,are-icons-missing,NaN,NaN,142,0,0


In [34]:
comp_kernels_df.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
604495,22725748,8659751,104435323.0,82118583.0,NaN,82121236.0,12/11/2021 15:02:31,08/29/2022,07/23/2022,False,hub-inference-single,NaN,NaN,250,0,5
736679,28160803,10654180,99073075.0,NaN,332690.0,99068065.0,06/22/2022 19:23:38,06/22/2022,06/22/2022,False,hubmap-quick-eda,2.0,09/04/2022,579,1,26
736684,28160898,1536542,99068725.0,NaN,342546.0,99068387.0,06/22/2022 19:30:46,06/22/2022,06/22/2022,False,background-information,2.0,07/14/2022,1949,0,41
736687,28161018,3894504,100803535.0,NaN,336792.0,99068759.0,06/22/2022 19:38:17,07/14/2022,06/26/2022,False,eda-simplified-hubmap-hpa-organ-segmentation,3.0,07/13/2022,207,2,9
736690,28161123,1314864,99071779.0,NaN,333397.0,99069092.0,06/22/2022 19:45:55,06/22/2022,06/22/2022,False,hubmap-hpa-exploratory-data-analysis,2.0,06/29/2022,560,6,36


In [35]:
target = comp_kernels_df[comp_kernels_df.CurrentUrlSlug=="unet-baseline-train"]
target

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes


In [36]:
target_parent = kernels_df[kernels_df.CurrentKernelVersionId==47074200]
target_parent


,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
431343,12930165,1094066,47074200.0,NaN,NaN,47065816.0,11/17/2020 08:28:26,11/17/2020,11/17/2020,False,hubmap-res34unet-baseline-train,2.0,11/29/2020,4668,0,30


In [37]:
#filter submissions
submissions_df = pd.read_csv(IN_DIR+'/Submissions.csv', infer_datetime_format=True)
submissions_df

,Id,SubmittedUserId,TeamId,SourceKernelVersionId,SubmissionDate,ScoreDate,IsAfterDeadline,PublicScoreLeaderboardDisplay,PublicScoreFullPrecision,PrivateScoreLeaderboardDisplay,PrivateScoreFullPrecision
0,184883,36583.0,23484,NaN,09/14/2012,NaN,False,0.67118,0.671184,0.00000,0.000000
1,184888,36583.0,23484,NaN,09/14/2012,NaN,False,0.66547,0.665475,0.00000,0.000000
2,184891,36583.0,23484,NaN,09/14/2012,NaN,False,0.67115,0.671152,0.00000,0.000000
3,184893,36583.0,23484,NaN,09/14/2012,NaN,False,0.66925,0.669260,0.00000,0.000000
4,184900,36583.0,23484,NaN,09/14/2012,NaN,False,0.65669,0.656698,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
11441799,28493214,11189420.0,9300197,NaN,10/16/2022,10/16/2022,True,93.15419,93.154193,78.84385,78.843851
11441800,28493131,11189420.0,9300197,NaN,10/16/2022,10/16/2022,True,75.33525,75.335251,62.78260,62.782605
11441801,28599920,11189420.0,9300197,NaN,10/24/2022,10/24/2022,True,92.92189,92.921897,80.15972,80.159727
11441802,28414176,9777342.0,9265889,NaN,10/10/2022,10/10/2022,True,63.98439,63.984399,47.11579,47.115797


In [38]:
comp_submissions_df = submissions_df[submissions_df.TeamId.isin(team_ids_list)]
comp_submissions_df

,Id,SubmittedUserId,TeamId,SourceKernelVersionId,SubmissionDate,ScoreDate,IsAfterDeadline,PublicScoreLeaderboardDisplay,PublicScoreFullPrecision,PrivateScoreLeaderboardDisplay,PrivateScoreFullPrecision
10938022,27177301,1107630.0,8805846,99083601.0,06/23/2022,06/23/2022,False,0.00017,0.000178,0.00016,0.000168
10938023,27188747,1107630.0,8805846,99083601.0,06/24/2022,06/24/2022,False,0.00017,0.000178,0.00016,0.000168
10938286,27271946,3045170.0,8806499,99739796.0,07/01/2022,07/01/2022,False,0.64465,0.644654,0.61432,0.614323
10938287,27425049,3215961.0,8806782,100725048.0,07/13/2022,07/13/2022,False,0.56559,0.565597,0.51279,0.512796
10938288,27499221,3215961.0,8806782,101119599.0,07/18/2022,07/18/2022,False,0.66756,0.667563,0.64493,0.644932
...,...,...,...,...,...,...,...,...,...,...,...
11437793,28090802,1422008.0,9146819,105567395.0,09/13/2022,09/13/2022,False,0.74504,0.745045,0.71778,0.717784
11437794,28090958,1422008.0,9146819,105568906.0,09/13/2022,09/13/2022,False,0.74504,0.745045,0.71778,0.717784
11441273,28126126,11605420.0,9156781,105816662.0,09/16/2022,09/16/2022,False,NaN,NaN,NaN,NaN
11441274,28126165,11605420.0,9156781,105817027.0,09/16/2022,09/16/2022,False,0.54423,0.544232,0.44717,0.447175


In [39]:
comp_submissions_df.to_csv(OUT_DIR+'/Submissions.csv')
